# 2D Inversion example

This example is based on the tutorial found at http://pysit.readthedocs.io/en/stable/quick_start/index.html  See the link for more details.  This example illustrates how to run pysit from a jupyter notebook within a docker container.

In [ ]:
# Set Up the Computing Environment
from pysit import *
import numpy as np
import matplotlib.pyplot as plt
from pysit.gallery import horizontal_reflector

In [ ]:
# Set up the domain
pmlx = PML(0.1, 100)
pmlz = PML(0.1, 100)
x_config = (0.1, 1.0, pmlx, pmlx)
z_config = (0.1, 0.8, pmlz, pmlz)
domain = RectangularDomain(x_config, z_config)
mesh = CartesianMesh(domain, 91, 71)
C, C0, mesh, domain = horizontal_reflector(mesh)



plt.figure()
vis.plot(C, mesh)
plt.draw()
plt.title('True Velocity Model')

In [ ]:
# Generate synthethic data

zmin = domain.z.lbound
zmax = domain.z.rbound
zpos = zmin + (1./9.)*zmax

shots = equispaced_acquisition(mesh,
                               RickerWavelet(10.0),
                               sources=1,
                               source_depth=zpos,
                               receivers='max',
                               receiver_depth=zpos)

solver = ConstantDensityAcousticWave(mesh,
                                     formulation='scalar',
                                     spatial_accuracy_order=2,
                                     trange=(0.0, 3.0),
                                     kernel_implementation='cpp')
wavefields = []
base_model = solver.ModelParameters(mesh, {'C': C})
generate_seismic_data(shots,
                      solver,
                      base_model,
                      wavefields=wavefields)

In [ ]:
# plot the seismogram
plt.figure()
vis.plot_seismogram(shots[0])

In [ ]:
# now do some FWI!

objective = TemporalLeastSquares(solver)
invalg = LBFGS(objective)
initial_value = solver.ModelParameters(mesh, {'C': C0})

status_configuration = {'value_frequency': 1,
                        'residual_frequency': 1,
                        'residual_length_frequency': 1,
                        'objective_frequency': 1,
                        'step_frequency': 1,
                        'step_length_frequency': 1,
                        'gradient_frequency': 1,
                        'gradient_length_frequency': 1,
                        'run_time_frequency': 1,
                        'alpha_frequency': 1}
nsteps = 5
result = invalg(shots,
                initial_value,
                nsteps,
                line_search='backtrack',
                status_configuration=status_configuration,
                verbose=True)

In [ ]:
# plot the results and inversion
clim = C.min(),C.max()
plt.figure()
plt.subplot(1,3,1)
vis.plot(C0, mesh, clim=clim)
plt.title('Initial Model')
plt.subplot(1,3,2)
vis.plot(C, mesh, clim=clim)
plt.title('True Model')
plt.subplot(1,3,3)
vis.plot(result.C, mesh, clim=clim)
plt.title('Reconstruction')
plt.draw()

In [ ]:
# plot the objective value
obj_vals = np.array([v for k,v in invalg.objective_history.items()])
plt.figure()
plt.semilogy(obj_vals)
plt.xlabel('iteration')
plt.ylabel('Objective value')
plt.show()